## anomaly_detection
for all exercises

## Discrete data + probability

#### Techniques for identifying/detecting anomalies
#### Statistical Methods

In [1]:
from __future__ import division

import itertools
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from numpy import linspace, loadtxt, ones, convolve
import pandas as pd
import math
from datetime import datetime
from dateutil.parser import parse

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn import metrics
from sklearn.ensemble import IsolationForest
from random import randint
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

import collections

# set default pandas decimal number display format and df display format:
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Wrangle Data

#### Acquire

In [2]:
colnames=['ip', 'timestamp', 'request_method', 'status', 'size',
          'destination', 'request_agent']
df_orig = pd.read_csv('http://python.zach.lol/access.log',          
                 engine='python',
                 header=None,
                 index_col=False,
                 names=colnames,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"',
                 usecols=[0, 3, 4, 5, 6, 7, 8])

In [3]:
new = pd.DataFrame([["95.31.18.119", "[21/Apr/2019:10:02:41+0000]", 
                     "GET /api/v1/items/HTTP/1.1", 200, 1153005, np.nan, 
                     "python-requests/2.21.0"],
                    ["95.31.16.121", "[17/Apr/2019:19:36:41+0000]", 
                     "GET /api/v1/sales?page=79/HTTP/1.1", 301, 1005, np.nan, 
                     "python-requests/2.21.0"],
                    ["97.105.15.120", "[18/Apr/2019:19:42:41+0000]", 
                     "GET /api/v1/sales?page=79/HTTP/1.1", 301, 2560, np.nan, 
                     "python-requests/2.21.0"],
                    ["97.105.19.58", "[19/Apr/2019:19:42:41+0000]", 
                     "GET /api/v1/sales?page=79/HTTP/1.1", 200, 2056327, np.nan, 
                     "python-requests/2.21.0"]], columns=colnames)

df = df_orig.append(new)

In [4]:
df.head()

,ip,timestamp,request_method,status,size,destination,request_agent
0,97.105.19.58,[16/Apr/2019:19:34:42 +0000],"""GET /api/v1/sales?page=81 HTTP/1.1""",200,512495,NaN,"""python-requests/2.21.0"""
1,97.105.19.58,[16/Apr/2019:19:34:42 +0000],"""GET /api/v1/items HTTP/1.1""",200,3561,NaN,"""python-requests/2.21.0"""
2,97.105.19.58,[16/Apr/2019:19:34:44 +0000],"""GET /api/v1/sales?page=82 HTTP/1.1""",200,510103,NaN,"""python-requests/2.21.0"""
3,97.105.19.58,[16/Apr/2019:19:34:46 +0000],"""GET /api/v1/sales?page=83 HTTP/1.1""",200,510003,NaN,"""python-requests/2.21.0"""
4,97.105.19.58,[16/Apr/2019:19:34:48 +0000],"""GET /api/v1/sales?page=84 HTTP/1.1""",200,511963,NaN,"""python-requests/2.21.0"""


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13978 entries, 0 to 3
Data columns (total 7 columns):
ip                13978 non-null object
timestamp         13978 non-null object
request_method    13978 non-null object
status            13978 non-null int64
size              13978 non-null int64
destination       25 non-null object
request_agent     13978 non-null object
dtypes: int64(2), object(5)
memory usage: 873.6+ KB


#### Parse Datetime

In [6]:
df.timestamp = df.timestamp.str.replace(r'(\[|\])', '', regex=True)
df.timestamp = pd.to_datetime(df.timestamp.str.replace(':', ' ', 1)) 
df = df.set_index('timestamp')

In [7]:
df.head()

,ip,request_method,status,size,destination,request_agent
timestamp,,,,,,
2019-04-16 19:34:42,97.105.19.58,"""GET /api/v1/sales?page=81 HTTP/1.1""",200,512495,NaN,"""python-requests/2.21.0"""
2019-04-16 19:34:42,97.105.19.58,"""GET /api/v1/items HTTP/1.1""",200,3561,NaN,"""python-requests/2.21.0"""
2019-04-16 19:34:44,97.105.19.58,"""GET /api/v1/sales?page=82 HTTP/1.1""",200,510103,NaN,"""python-requests/2.21.0"""
2019-04-16 19:34:46,97.105.19.58,"""GET /api/v1/sales?page=83 HTTP/1.1""",200,510003,NaN,"""python-requests/2.21.0"""
2019-04-16 19:34:48,97.105.19.58,"""GET /api/v1/sales?page=84 HTTP/1.1""",200,511963,NaN,"""python-requests/2.21.0"""


In [8]:
df['destination'].unique()

array([nan, '"https://python.zach.lol/api/V1/HiZach!"',
       '"https://python.zach.lol/api/v1/stores?page=0"',
       '"https://python.zach.lol/api/v1/stores?page=1"',
       '"https://python.zach.lol/api/v1/stores?page=2"',
       '"https://python.zach.lol/api/v1/stores?page=999"',
       '"https://python.zach.lol/api/v1/items?page=0"',
       '"http://localhost:8889/notebooks/timeseries_acquisition.ipynb"',
       '"https://python.zach.lol/api/v1//api/v1/items?page=2"',
       '"https://python.zach.lol/api/v1//api/v1/items"',
       '"https://python.zach.lol/api/v1//api/v1/items/next_page"',
       '"https://python.zach.lol/api/v1/helloclass!"',
       '"https://python.zach.lol/api/v1/I_DIDNT_DO_IT!!!!"',
       '"http://localhost:8888/notebooks/acquire.ipynb"',
       '"https://python.zach.lol/api/v1/sales?page=3"',
       '"https://ds.codeup.com/8.3_Acquire/"',
       '"https://python.zach.lol/"',
       '"https://python.zach.lol/api/v1/items"',
       '"https://python.zach.lol/a

In [9]:
df.request_method.nunique()

220

#### Clean up text

These two steps below help to "normalize" the data by removing a bunch of distinct but unnecessary items, like the page numbers etc.  This will reduce the MANY distinct items down to a handful of distinct items.  Makes this easier, and REDUCES THE NOISE!

In [10]:
for col in ['request_method', 'request_agent', 'destination']:
    df[col] = df[col].str.replace('"', '')

In [11]:
df['request_method'] = df.request_method.str.replace(r'\?page=[0-9]+', '', regex=True)
# this finds the "question mark\page=number" and this finds the literal word "page", and then find the equal sign,
# then find a numerical value, and it's repeated a number of times, hence the plus sign.

In [12]:
df.head(10)

,ip,request_method,status,size,destination,request_agent
timestamp,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET /api/v1/sales HTTP/1.1,200,512495,NaN,python-requests/2.21.0
2019-04-16 19:34:42,97.105.19.58,GET /api/v1/items HTTP/1.1,200,3561,NaN,python-requests/2.21.0
2019-04-16 19:34:44,97.105.19.58,GET /api/v1/sales HTTP/1.1,200,510103,NaN,python-requests/2.21.0
2019-04-16 19:34:46,97.105.19.58,GET /api/v1/sales HTTP/1.1,200,510003,NaN,python-requests/2.21.0
2019-04-16 19:34:48,97.105.19.58,GET /api/v1/sales HTTP/1.1,200,511963,NaN,python-requests/2.21.0
2019-04-16 19:34:48,97.105.19.58,GET /api/v1/stores HTTP/1.1,200,1328,NaN,python-requests/2.21.0
2019-04-16 19:34:50,97.105.19.58,GET /api/v1/sales HTTP/1.1,200,510753,NaN,python-requests/2.21.0
2019-04-16 19:34:52,97.105.19.58,GET /api/v1/sales HTTP/1.1,200,510348,NaN,python-requests/2.21.0
2019-04-16 19:34:52,97.105.19.58,GET / HTTP/1.1,200,42,NaN,python-requests/2.21.0


In [13]:
df.request_method.nunique()

22

#### Add variable: converting bytes to mb

In [14]:
df['size_mb'] = [n/1024/1024 for n in df['size']]
df.describe()

,status,size,size_mb
count,"13,978.00","13,978.00","13,978.00"
mean,200.36,"450,001.91",0.43
std,10.18,"161,491.47",0.15
min,200.00,0.00,0.00
25%,200.00,"500,637.00",0.48
50%,200.00,"510,138.00",0.49
75%,200.00,"511,291.00",0.49
max,499.00,"2,056,327.00",1.96


In [15]:
df.request_method.value_counts(dropna=False)

GET /api/v1/sales HTTP/1.1                      12403
GET /api/v1/items HTTP/1.1                       1065
GET /api/v1/stores HTTP/1.1                       229
GET / HTTP/1.1                                    107
GET /documentation HTTP/1.1                       100
GET /favicon.ico HTTP/1.1                          26
GET /api/v1//api/v1/items HTTP/1.1                 11
GET /api/v1/items/api/v1/items HTTP/1.1             7
GET /api/v1/items/next_page HTTP/1.1                5
GET /api/v1/ HTTP/1.1                               4
GET /api/v1/sales/HTTP/1.1                          3
GET /api/v1/sales/ HTTP/1.1                         3
GET /api/v1/store HTTP/1.1                          3
GET /api/v1/itemsitems HTTP/1.1                     3
GET /api/v1items HTTP/1.1                           2
GET /api/v1/items&page=0 HTTP/1.1                   1
GET /api/V1/HiZach! HTTP/1.1                        1
GET /api/v1/I_DIDNT_DO_IT!!!! HTTP/1.1              1
GET /api/v1/helloclass! HTTP

In [16]:
df.request_method.value_counts(dropna=False)/df.request_method.count()

GET /api/v1/sales HTTP/1.1                                     0.89
GET /api/v1/items HTTP/1.1                                     0.08
GET /api/v1/stores HTTP/1.1                                    0.02
GET / HTTP/1.1                                                 0.01
GET /documentation HTTP/1.1                                    0.01
GET /favicon.ico HTTP/1.1                                      0.00
GET /api/v1//api/v1/items HTTP/1.1                             0.00
GET /api/v1/items/api/v1/items HTTP/1.1                        0.00
GET /api/v1/items/next_page HTTP/1.1                           0.00
GET /api/v1/ HTTP/1.1                                          0.00
GET /api/v1/sales/HTTP/1.1                                     0.00
GET /api/v1/sales/ HTTP/1.1                                    0.00
GET /api/v1/store HTTP/1.1                                     0.00
GET /api/v1/itemsitems HTTP/1.1                                0.00
GET /api/v1items HTTP/1.1                       

In [17]:
df.request_agent.value_counts(dropna=False)/df.request_agent.count()

python-requests/2.21.0                                                                                                                      0.86
python-requests/2.20.1                                                                                                                      0.14
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36                   0.00
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0                                                          0.00
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)                                                                                  0.00
Slackbot 1.0 (+https://api.slack.com/robots)                                                                                                0.00
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36         

### Detecting Anomalies in Discrete Variables
#### Finding anomalies in already existing data
We can see easily some anomalies around various request_agents.

In [18]:
agent_df = pd.DataFrame(df.request_agent.value_counts(dropna=False)).reset_index().\
                       rename(index=str, columns={'index': 'request_agent','request_agent':'agent_count'})

agent_df2 = pd.DataFrame(df.request_agent.value_counts(dropna=False)/df.request_agent.count()).reset_index().\
                       rename(index=str, columns={'index': 'request_agent','request_agent':'agent_proba'})
agent_df = agent_df.merge(agent_df2)

In [19]:
agent_df.shape

(9, 3)

In [20]:
agent_df.head(20)

,request_agent,agent_count,agent_proba
0,python-requests/2.21.0,12005,0.86
1,python-requests/2.20.1,1911,0.14
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,34,0.00
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,8,0.00
4,Slackbot-LinkExpanding 1.0 (+https://api.slack...,7,0.00
5,Slackbot 1.0 (+https://api.slack.com/robots),6,0.00
6,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,4,0.00
7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,2,0.00
8,Python-urllib/3.7,1,0.00


In [21]:
# see those where rate < 1% 
agent_df[agent_df.agent_proba < .01]

,request_agent,agent_count,agent_proba
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,34,0.00
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,8,0.00
4,Slackbot-LinkExpanding 1.0 (+https://api.slack...,7,0.00
5,Slackbot 1.0 (+https://api.slack.com/robots),6,0.00
6,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,4,0.00
7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,2,0.00
8,Python-urllib/3.7,1,0.00


In [22]:
print('Length of df:')
print(len(agent_df))
print('different distinct request_agents pinging the Zach_lol server.')
print('\n')
print('Tail of df:')
print('... the probabilities of those request_agents pinging that server:')
print('\n')
print(agent_df.tail(10))
print('\n')
print('Combining the first two request_agent probabilities, this means greater than 99.55% of the probability of hitting this server falls into the two types of python requests, 2.21 and 2.20.\n')
print('\n')
print('This means, most anything hitting this server that is NOT a python 2.20 and 2.21 request, is likely an anomaly or an outlier, for good or bad.')


Length of df:
9
different distinct request_agents pinging the Zach_lol server.


Tail of df:
... the probabilities of those request_agents pinging that server:


                                       request_agent  agent_count          agent_proba
0                             python-requests/2.21.0        12005                 0.86
1                             python-requests/2.20.1         1911                 0.14
2  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...           34                 0.00
3  Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...            8                 0.00
4  Slackbot-LinkExpanding 1.0 (+https://api.slack...            7                 0.00
5       Slackbot 1.0 (+https://api.slack.com/robots)            6                 0.00
6  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...            4                 0.00
7  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...            2                 0.00
8                                  Python-urllib/3.7   

### Plotting the request_agent numbers:

In [23]:
plt.figure(figsize=(12,4))
splot = sns.barplot(data=agent_df, x = 'request_agent', y = 'agent_count', ci = None)
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.0f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', xytext = (0, 10), 
                   textcoords = 'offset points'
                   )
    plt.xticks(rotation='vertical')

NameError: name 'sns' is not defined

<Figure size 864x288 with 0 Axes>

### Detecting anomalies by establishing a baseline and evaluate as new data arrives
#### Establish baseline

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
train = df['2019-04-16 19:34:42':'2019-04-17 12:55:14'][['ip','request_method','status','size','destination','request_agent','size_mb']]

#### Compute probabilities based on train sample

In [ ]:
agent_df = pd.DataFrame(train.request_agent.value_counts(dropna=False)/train.request_agent.count()).reset_index().\
                rename(index=str, columns={'index': 'request_agent', 'request_agent': 'agent_proba'})

In [ ]:
agent_df.head(20)

### Merge probabilities with all data (train + new data)
#### Where the ip address is new, i.e. not seen in the training dataset, fill the probability with a value of 0.

In [ ]:
df = df.reset_index().merge(agent_df, on=['request_agent'], how='left').fillna(value=0).set_index('timestamp')

In [ ]:
df.head()

In [ ]:
df.agent_proba.value_counts()

### Conditional Probabilities: probabilities using 2 discrete variables
#### Probability of destination given request_agent:
If we are looking for an unexpected destination (like weird website, etc) from a known/common request agent.

In [ ]:
train.groupby('request_agent').size()
# this is the count by request_agent

In [ ]:
train.groupby('request_agent').size().div(len(df))
# this is the probability of the distinct request_agent items by the total data frame

In [ ]:
# now dividing the size, above, by the length of the df
# This is a repeat of the above cell, except setting a new data frame with these items and probs:
request_probs = train.groupby('request_agent').size().div(len(df))
request_probs

In [ ]:
destination_given_agent = pd.DataFrame(train.groupby(['request_agent', 'destination']).\
                               size().div(len(train)).\
                               div(request_probs, 
                                   axis=0, 
                                   level='request_agent').\
                               reset_index().\
                               rename(index=str, 
                                      columns={0: 'proba_destination_given_agent'})
                              )

In [ ]:
destination_given_agent.head(40)

In [ ]:
agent_destination_count = pd.DataFrame(train.groupby(['request_agent', 'destination'])['request_method'].\
                                count().reset_index().\
                                rename(index=str, 
                                       columns={'request_method': 'agent_destination_count'}))

In [ ]:
agent_destination_count.agent_destination_count.nunique()

In [ ]:
df.request_method.unique()

In [ ]:
agent_destination = destination_given_agent.merge(agent_destination_count)

In [ ]:
agent_destination.head()

In [ ]:
plt.plot(agent_destination.proba_destination_given_agent)

### Add these probabilities to original events to detect anomalous events

In [ ]:
df = df.reset_index().merge(agent_destination, on=['request_agent', 'destination'], how='left').fillna(value=0).set_index('timestamp')

In [ ]:
df.head()

In [ ]:
plt.scatter(df.proba_destination_given_agent, df.agent_proba)

## Time series + EMA

Discover users who are accessing our curriculum pages way beyond the end of their codeup time.
- What would the dataframe look like?
- Use time series method for detecting anomalies, like exponential moving average with %b.
- See (paper) notes for good details on how to do this, to include graphics and images, as well as the previous lesson, named:
- ("anomaly_detection_lesson_29Apr19_timeseries_anomalies.ipynb" in same folder)

## Review Maggie's walkthrough here... it calculates number of distinct users w/page view per day per cohort.

#### she calculates the following:
over 7 days

for each cohort per day:
- EMA
- upper bound (ub)
- lower bound (lb)
- actual value
- % bound

(current - lb) / divided by / (ub - lb)

#### Bringing in data from csv

In [ ]:
colnames=['date', 'page_viewed', 'user_id', 'cohort', 'ip']
df_orig = pd.read_csv('/Users/rachelreuter/ds-methodologies/anomaly_detection/anonymized-curriculum-access.txt',
    header=None,
    index_col=False,
    names=colnames,
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
    na_values='"-"',
    usecols=[0,2,3,4,5]
)

In [ ]:
df_orig.shape

In [ ]:
df_orig.head()

### Below, keep this code to filter on multiple conditions:

In [ ]:
df_monkey = df_orig

In [ ]:
df_monkey = df_monkey[(df_monkey.cohort.isnull()) & (df_monkey.date > '2018-04-01') & (df_monkey.user_id == 366)]

In [ ]:
df_monkey.shape

## Keep this snippet below to spin off a separate copy of the df, then group and aggregate.
#### It creates a new dataframe using a copy of the original dataframe, and brings over only two fields (user_id and date).
- .copy() is mandatory here, to avoid affecting the original df, because when it says 'df_b = df_a', it means it is connected.

#### Then the next step groups on the user id, and then grabs the min and max from the date column, and creates two additional columns for the min and max log in time for each user for each day.

### After an aggegation, if the column headers end up on different levels:

### ... now back to the actual code in this lesson...

#### Now, perform slight manipulations on the df_orig, drop na rows and converting 'cohort' field to integer:

In [ ]:
df_orig = df_orig.dropna()
df_orig.cohort = df_orig.cohort.astype('int')
print(df_orig.info())
df_orig.head()

#### Bringing in the Codeup cohort names, for use later

In [ ]:
colnames=['cohort_name','cohort']
df_cohort = pd.read_csv('cohort_id_name.txt',
                       names=colnames, 
                       skiprows=1)
print(df_cohort.info())
df_cohort.head(32)

#### Merging the Codeup cohort names with the df, into one df.

In [ ]:
df = df_orig.merge(df_cohort, on='cohort', how='left')
df.head()

#### Looking at shape, columns, dtypes and description of data

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
#df.dtypes

In [ ]:
df.describe()

#### Combining date and time into one field

In [ ]:
# df_orig['dateandtime'] = df_orig['date'] + ' ' + df_orig['time']

In [ ]:
# df_orig.head()

- Success

#### Changing that new combined dateandtime field to datetime format

In [ ]:
df['date'] = pd.to_datetime(df.date)
df = df.dropna()

In [ ]:
# df_orig['dateandtime'] = pd.to_datetime(df_orig['dateandtime'])

In [ ]:
#df_orig['date'] = pd.to_datetime(df_orig['date'])

In [ ]:
df.dtypes

In [ ]:
df.shape

- Success

#### Checking the dataframe for blanks, nulls, missing, Nan, etc

In [ ]:
def missing_values_col(df):
	"""
	Write or use a previously written function to return the
	total missing values and the percent missing values by column.
	"""
	null_count = df.isnull().sum()
	null_percentage = (null_count / df.shape[0]) * 100
	empty_count = pd.Series(((df == ' ') | (df == '')).sum())
	empty_percentage = (empty_count / df.shape[0]) * 100
	nan_count = pd.Series(((df == 'nan') | (df == 'NaN')).sum())
	nan_percentage = (nan_count / df.shape[0]) * 100
	return pd.DataFrame({'num_missing': null_count, 'missing_percentage': null_percentage,
	                     'num_empty': empty_count, 'empty_percentage': empty_percentage,
	                     'nan_count': nan_count, 'nan_percentage': nan_percentage})

In [ ]:
missing_values_col(df)

In [ ]:
def peekatdata(df):
    print("\n \n SHAPE:")
    print(df.shape)

    print("\n \n COLS:")
    print(df.columns)

    print("\n \n INFO:")
    print(df.info())

    print("\n \n Missing Values:")
    missing_vals = df.columns[df.isnull().any()]
    print(df.isnull().sum())

    print("\n \n DESCRIBE:")
    print(df.describe())

    print('\n \n HEAD:')
    print(df.head(5))

    print('\n \n TAIL:' )
    print(df.tail(5))

In [ ]:
peekatdata(df)

#### Filling in any Nans with zero

In [ ]:
#df_orig = df_orig.fillna(0)

In [ ]:
#df_orig[['cohort']] = df_orig[['cohort']].astype(int)

In [ ]:
#df_orig[['cohort']] = df_orig[['cohort']].astype(str)

In [ ]:
#df_orig[['student_id']] = df_orig[['student_id']].astype(str)

In [ ]:
# df_orig = field_drop(df_orig)

In [ ]:
#df_orig = df_orig[['date','dateandtime','ip','cohort','student_id','curriculum_topic']]

##### Dataframe now appears to be ready to review.  Will check with .describe.

In [ ]:
df.describe(include='all')

##### Dataframe is prepared and ready to proceed with the anomaly detection lesson.

## Section below is specifically preparing the data so we can look at the EMA and %B.

#### Reiterate the lesson objective:

Discover users who are accessing our curriculum pages way beyond the end of their codeup time.
- What would the dataframe look like?
- Use time series method for detecting anomalies, like exponential moving average with %b.
- See (paper) notes for good details on how to do this, to include graphics and images, as well as the previous lesson:
- ("anomaly_detection_lesson_29Apr19_timeseries_anomalies.ipynb" in same folder)

#### Creating a new AGGREGATED dataframe grouped by date, cohort and cohort name, and counting instances of user IDs, for that combination of cohort and cohort name ON THAT DATE.

## IMPORTANT!
- Note, groupby code below is LIKE A PIVOT TABLE, where the group by fields are the "rows" in the pivot, and the field in the list immediately following is the "values" section of a pivot table.
- The 'nunique()' aggregation immediately following is like the 'sum, count, average' math function in the values part of the pivot.

In [ ]:
df_agg = df.groupby(['date','cohort','cohort_name'])['user_id'].\
nunique().\
reset_index().\
rename(index=str, columns={'user_id': 'users_viewed'})


# df_agg = df.groupby(['date','cohort_id','cohort_name'])['user_id'].\
#                         nunique().\
#                     reset_index().\
#                     rename(index=str, 
#                        columns={'user_id': 'users_viewed'})

In [ ]:
df_agg.tail()

In [ ]:
df_agg.shape

In [ ]:
df_agg.info()

#### she calculates the following:
over 7 days

for each cohort per day:
- EMA
- upper bound (ub)
- lower bound (lb)
- actual value
- % bound

(current - lb) / divided by / (ub - lb)

### EMA

Exponential Moving Average (EMA) helps reduce the lag induced by the use of the SMA, or Simple Moving Average. It does this by putting more weight on more recent observations, whereas the SMA weights all observations equally.

#### First, create separate list of Codeup class cohorts to use later

In [ ]:
cohorts = list(df_agg.cohort_name.unique())
cohorts

#### Then, set the index to date on the AGGREGATED df, and rename 'users_viewed' field to 'ema', or 'Exponential Moving Average'.

Again, the Exponential Moving Average (EMA) helps reduce the lag induced by the use of the SMA, or Simple Moving Average. It does this by putting more weight on more recent observations, whereas the SMA weights all observations equally.

In [ ]:
df = df_agg.set_index('date').\
rename(index=str, columns={'users_viewed':'ema'}).\
drop(columns='cohort')

# df = df_agg.set_index('date').\
#             rename(index=str, columns={'users_viewed':'ema'}).\
#             drop(columns='cohort_id')

In [ ]:
df.head()

#### Now create the Bollinger bands, and setting a standard deviation rolling average (EMA, see further below) of 14 days:

Bollinger bands are created by multiplying plus or minus three standard deviations.  The upper bounds and lower bounds are created from the standard deviation.  For lag days inside that initial 14-day rolling window, nans will initially be created, but will be filled in later... see code later for further details.

In [ ]:
def bollinger_bands(df):
    ema = df.ewm(span=14, adjust=False).mean()
    ema['stdev'] = ema.ema.rolling(14).std()
    ema['ub'] = ema.ema + ema.stdev * 3
    ema['lb'] = ema.ema - ema.stdev * 3
    return ema.reset_index()

In [ ]:
# ema = df[df.cohort_name == 'Arches'].ewm(span=14, adjust=False).mean()
# ema['stdev'] = ema.ema.rolling(14).std()
# ema['ub'] = ema.ema + ema.stdev * 3
# ema['lb'] = ema.ema - ema.stdev * 3
# ema.head(20)

#### Now create new 'bb' df by calling that bollinger_bands() function on the original 'df', where the df.cohort_name matches the list of cohort names, created just above this cell, at the start of the "EMA" section.

#### Now feed that bb df into an empty list called bands... then concat that list into a new df called 'df2'

In [ ]:
bands = []
for cohort in cohorts:
    bb = bollinger_bands(df[df.cohort_name == cohort])
    bands.append(bb)

df2 = pd.concat(bands)

# bands is a list of dfs, one df for each cohort name


# bands = []
# for cohort in cohorts:
#     bb = bollinger_bands(df[df.cohort_name == cohort])
#     bands.append(bb)

# df2 = pd.concat(bands)

In [ ]:
df2.head(20)

#### Notice the stdev, ub and lb fields in df2 immediately above.  Check for null or nan values:

In [ ]:
df2.isnull().sum()

#### Now start to deal with those nans by creating yet another df containing the fields 'cohort_name' and 'ema' from df2, by looking at df2, specifically where the stdev field in df2 is null.

In [ ]:
df_missing = df2[df2.stdev.isnull()][['cohort_name','ema']]
df_missing.head(22)

#### Now group the new df_missing df on field 'cohort_name', and fill in the missing stdevs with zero, reset the index, and finally, rename the 'ema' field to 'stdev_null'.

#### This groups/aggs the stdevs for all items in each cohort name.

#### ...And creates a stdev for any cohort/day where the stdev may be missing, due to the lag days involved in using EMA, in this case 14 days.

In [ ]:
df_missing = df_missing.groupby('cohort_name').std().fillna(value=0).reset_index().rename(index=str, columns={'ema': 'stdev_null'})

In [ ]:
df_missing.head(31)

#### Now merge the df_missing immediately above (containing the 'missing' stdev values for each cohort) with the df2 from further above, containing the Bollinger Bands for each cohort, for each date.

#### Merge this into a new df, merging on 'cohort_name'.

In [ ]:
df = df2.merge(df_missing, on='cohort_name', how='left')

In [ ]:
df.tail(15)

#### Now, use idx as the index to find where in the df the stdev field is null.
 
#### Then after locating those null rows/records in the stdev field, now fill them with the value in the stdev_null field that we just created above.  Then, afterward, drop the stdev_null field, as it's no longer needed.

In [ ]:
idx = df.stdev.isnull()
df.loc[idx,'stdev'] = df.loc[idx,'stdev_null']
df = df.drop(columns='stdev_null')

In [ ]:
df.head()

#### Note, the nans in the stdev field above have been filled in accordingly.

#### Now, same routine with the upper and lower bound fields.  These will be filled in with the stdev-calculated formulas shown below:

#### First locate the null values in the ub field and label with idx.  Then use idx to locate and populate both ub and lb nulls accordingly.

In [ ]:
idx = df.ub.isnull()
df.loc[idx,'ub'] = df.loc[idx,'ema'] + df.loc[idx,'stdev']*3
df.loc[idx,'lb'] = df.loc[idx,'ema'] - df.loc[idx,'stdev']*3

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df.isnull().sum()

#### Note, the nulls in stdev, ub and lb in df above are all populated... moving on to next steps.

Some ub and lb values may be equal in a given row if the ema is equal to 1.  See below for example.

This is an error and must be fixed:

In [ ]:
df[df.ub == df.lb].count()

In [ ]:
df[df.ub == df.lb]

#### This is an error and must be fixed.

#### First locate those rows, then nudge each up or down accordingly (if ub or lb, respectively) by .01 to correct.

In [ ]:
idx = df.ub == df.lb
df.loc[idx,'ub'] = df.loc[idx,'ub'] + .01
df.loc[idx,'lb'] = df.loc[idx,'lb'] - .01

##### Now verify the fix worked.

In [ ]:
df[df.ub == df.lb].count()

##### Issue above has been fixed.

#### Now reset the date field in the df to datetime datatype:

In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
df.info()

#### Remember the AGGREGATED df_agg df from above, that contained groupings by date, cohort and cohort name, and counted instances of user IDs, for that combination of cohort and cohort name ON THAT DATE?

#### Let's pull it back up and look at its info... in preparation for merging.

In [ ]:
df_agg.info()

#### Now merge the df from above with the aggregated df_agg immediately above:

In [ ]:
# join with aggregated dataset to get the original count of users viewed
df = df.merge(df_agg, on=['cohort_name','date'], how='left')

In [ ]:
df.head()

#### Now compute the %b:

In [ ]:
# compute %b
df['pct_b'] = (df.users_viewed - df.lb)/(df.ub - df.lb)

In [ ]:
df = df.drop(columns=['stdev','cohort'])

In [ ]:
df.head()

#### Now sort all pct_b values "descending", looking for outliers above the 1.0 ub.

In [ ]:
df[df.pct_b>1].sort_values(by='pct_b', ascending=False)

#### I can do the same with those pct_b values below the 0.0 lb.

In [ ]:
df[df.pct_b<0].sort_values(by='pct_b', ascending=True)

#### Now plot out how this looks, with a rolling 7-day exponential moving average (EMA):

In [ ]:
fig = plt.figure(figsize=(16,16))
sns.lineplot(df.date, df.ema, hue=df.cohort_name)


# fig = plt.figure(figsize=(16,16))
# sns.lineplot(df.date, df.ema, hue=df.cohort_name)

## Clustering - DBSCAN

### Detect Anomalies using Density Based Clustering
Clustering-Based Anomaly Detection

- Use dbscan to detect anomalies in other products from the customers dataset (see this same directory).
- Use dbscan to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror) (ie zillow data).  for this one, see further below.

#### Prepare environment

See first cell above for the imports, tools, etc.

### Data Dictionary
#### *Attribute Information:*

1.  FRESH: annual spending (m.u.) on fresh products (Continuous); 
2.	MILK: annual spending (m.u.) on milk products (Continuous); 
3.	GROCERY: annual spending (m.u.)on grocery products (Continuous); 
4.	FROZEN: annual spending (m.u.)on frozen products (Continuous);
5.	DETERGENTS_PAPER: annual spending (m.u.) on detergents and paper products (Continuous);
6.	DELICATESSEN: annual spending (m.u.)on and delicatessen products (Continuous);
7.	CHANNEL: customersâ€™ Channel - Horeca (Hotel/Restaurant/CafÃ©) or Retail channel (Nominal);
8.	REGION: customersâ€™ Region â€“ Lisnon, Oporto or Other (Nominal).

#### *Descriptive Statistics:*

*(Minimum, Maximum, Mean, Std. Deviation)*
- FRESH: (	3, 112151, 12000.30, 12647.329) 
- MILK:	(55, 73498, 5796.27, 7380.377) 
- GROCERY:	(3, 92780, 7951.28, 9503.163) 
- FROZEN:	(25, 60869, 3071.93, 4854.673) 
- DETERGENTS_PAPER: (3, 40827, 2881.49, 4767.854) 
- DELICATESSEN: (3, 47943, 1524.87, 2820.106) 

REGION	Frequency
- Lisbon	77
- Oporto	47
- Other Region	316
- Total	440 

CHANNEL	Frequency 
- Horeca	298
- Retail	142
- Total	440

In [ ]:
# import csv and convert to a df
df = pd.read_csv('customers.csv')

In [ ]:
print('Shape:')
print(df.shape)
print('\nData Types:\n')
print(df.dtypes)
print('\nHead or tail view:')
df.tail(2)

### First, select any two numeric variables:

#### Grab only the Grocery, Milk and Fresh columns:

In [ ]:
df = df[['Grocery','Milk', 'Fresh']]

In [ ]:
print('Shape:')
print(df.shape)
print('\nData Types:\n')
print(df.dtypes)
print('\nHead or tail view:')
df.tail(2)

In [ ]:
df.head(10)

### Convert to numpy array (np.array) and set datatype to float:

In [ ]:
np_array = df.values.astype('float32', copy=False)

In [ ]:
np_array[:10]

### Normalize data:
- Normalize each attribute by scaling it to 0 mean and unit variance.
- This helps to keep the inter-relationships between the features intact so that a small change in one feature would reflect in the other.

    *How?*

    Use a scaler.  In this case, use Standard Scaler.

In [ ]:
stscaler = StandardScaler().fit(np_array)
np_array = stscaler.transform(np_array)

In [ ]:
np_array[0:10]

### Construct a DBSCAN object.

#### *...a DBSCAN object that requires a minimum of 15 data points in a neighborhood of radius 0.5 to be considered a core point.*

In [ ]:
dbsc = DBSCAN(eps = .75, min_samples = 15).fit(np_array)

*... this above constucts the DBSCAN object, and then fits it to the standardized np_array created above.*

### Next, extract our cluster labels and outliers to plot our results.

In [ ]:
labels = dbsc.labels_
labels[0:10]

*... this is a list of zeros and -1's, where 0 = "in the cluster," and -1 = "outside the cluster," for whatever reason.*

#### Below adds the labels to the original df, to analyze and explore.

In [ ]:
df['labels'] = labels
df.labels.value_counts()

# df['labels'] = labels
# df.labels.value_counts()

*... above, we have fed the"labels" array into a new "labels" field in the df, and then done a simple count of values.  This means 40 points are outside the cluster.  These may be anomalies.  Requires investigation.*

#### So, let's investigate those possible anomalies:

In [ ]:
df[df.labels == -1].head()

### Now, plot a couple fields against one another:

*Here is Grocery vs Fresh*

In [ ]:
sns.scatterplot(df.Grocery, df.Fresh, hue=df.labels)
plt.show()

*And here is Milk vs Fresh*

In [ ]:
sns.scatterplot(df.Milk, df.Fresh, hue=df.labels)
plt.show()

### And here is that view in 3D:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(1, figsize=(8, 8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

# plot the points
ax.scatter(df.Fresh, df.Milk, df.Grocery,
           c=df.labels, edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

ax.set_xlabel('Fresh')
ax.set_ylabel('Milk')
ax.set_zlabel('Grocery')

### Moving on to the next question in this lesson:

- Use dbscan to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).

### Repeat all the steps above, using the zillow data instead

### Repeat all the steps above a third time and maybe use the cvi data I brought in today

In [ ]:
prepare environment

In [ ]:
bring in the zillow data

In [ ]:
prep it like above

## Below is the work I was working through, prior to the walk through...

#### Note, MANY of the initial steps were removed and replaced with steps above.  The steps below are after all the manipulations to the df were done, and I was attempting to start grouping/aggregating, but having trouble.  

####  Just pick one cohort to do the grouping on, and then do the timeseries on that one grouped cohort.

#### Resample to 30 minute intervals taking min of curriculum page visited

In [ ]:
monkey = df.groupby([df.index.date,'student_id']).count()

In [ ]:
monkey

In [ ]:
monkey2 = df['student_id'].resample('1D').count()

In [ ]:
df.groupby('student_id').size()

In [ ]:
monkey2

In [ ]:
df.groupby('curriculum_topic').describe()

In [ ]:
df.groupby(df.index.date).agg(['min','max','mean'])

In [ ]:
#df = df['curriculum_topic'].resample('30T').min()
df2 = df['curriculum_topic'].resample('30T').value_counts.min()

In [ ]:
df2

In [ ]:
idx = pd.date_range(
    df2.sort_index().index.min(), 
    df2.sort_index().index.max(),
    freq='30min'
)

In [ ]:
df2 = df2.reindex(idx, fill_value=0).fillna(value=0)

In [ ]:
df2

### This is preparing the data so we can look at the avg, SMA, EMA and %B.


In [ ]:
start_date_train = df2.head(1).index[0]

In [ ]:
end_date_train = '2019-03-15 23:30:00'

In [ ]:
start_date_test = '2019-03-16 00:00:00'

In [ ]:
train = df2[:end_date_train]
test = df2[start_date_test:]

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(train)
plt.plot(test)
plt.show()

In [ ]:
# calculating upper and lower bands, Maggie provided in the slack channel:

span = 24
ema_long = train.ewm(span=span, adjust=False).mean()
midband = ema_long[-1]
ub = midband + ema_long[-24:-1].std()*3
lb = midband - ema_long[-24:-1].std()*3

yhat['moving_avg_forecast'] = midband